<a href="https://colab.research.google.com/github/mathav95raj/bird-event-detection-using-nmf/blob/master/birdeventdetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from scipy import signal
import librosa
from sklearn import  ensemble
from sklearn import metrics
import os

In [34]:

path = "/content/drive/My Drive/Torrent/ff1010bird/wav"

In [35]:
datalist = pd.read_csv("/content/drive/My Drive/Torrent/ff1010bird/ff1010bird_metadata.csv")
datalist

,itemid,hasbird
0,64486,0
1,2525,0
2,44981,0
3,101323,0
4,165746,0
...,...,...
7685,168059,0
7686,164922,0
7687,80789,1
7688,104733,1


In [36]:

print("bird wav", datalist[datalist['hasbird']==1].shape[0])
print("non bird wav", datalist[datalist['hasbird']==0].shape[0])

bird wav 1935
non bird wav 5755


In [37]:
data_work, data_nonwork = train_test_split(datalist, test_size=0.50, random_state=42)

In [38]:
data_train, data_test = train_test_split(data_work, test_size=0.20, random_state=42)

In [39]:
print("bird wav", data_train[data_train['hasbird']==1].shape[0])
print("non bird wav", data_train[data_train['hasbird']==0].shape[0])

bird wav 782
non bird wav 2294


In [40]:
id0_train = data_train[data_train['hasbird']== 0]
id1_train = data_train[data_train['hasbird']== 1]

In [41]:
id0_train_labels = id0_train.iloc[:,1].values.reshape(-1,1)
id1_train_labels = id1_train.iloc[:,1].values.reshape(-1,1)

In [42]:
win     = 1024
fs      = 44100.
n_mels  = 40
n_sh    = 4    
ham_win = np.hamming(win) 

In [43]:
melW = librosa.filters.mel(fs, n_fft=win, n_mels=n_mels, fmin=0., fmax=22100 )
melW /= np.max(melW, axis=-1)[:,None]  #normalizing mel filter across freq axis = 1, melW = no of filters X frequency bands

In [44]:
melW.shape

(40, 513)

In [45]:
id0_train_array_list = []
for index, row in id0_train.iterrows(): 
    wavfile = path + os.sep + str(row["itemid"]) + ".wav"
    wavarray, sr = librosa.core.load(wavfile, mono=True)
    if (wavarray.ndim==2): 
        wavarray = np.mean( wavarray, axis=-1 )    
    [fq, t, X] = signal.spectral.spectrogram( wavarray, window=ham_win, nperseg=win, noverlap=0, detrend=False, return_onesided=True, mode='magnitude' ) 
    X = X.T
    X = np.dot( X, melW.T )
    X=X/np.max(X) 
    id0_train_array_list.append(X)

In [46]:
X

array([[0.02208344, 0.06798306, 0.05658229, ..., 0.007633  , 0.00675879,
        0.00734073],
       [0.01370199, 0.05782418, 0.05832353, ..., 0.00618839, 0.00545592,
        0.0047037 ],
       [0.01301916, 0.08530617, 0.06895193, ..., 0.00542532, 0.00653322,
        0.00718341],
       ...,
       [0.02440473, 0.06883584, 0.04325444, ..., 0.0064768 , 0.00644982,
        0.00495907],
       [0.02366108, 0.05715118, 0.04496249, ..., 0.01183673, 0.01087546,
        0.00650025],
       [0.0106283 , 0.03947097, 0.04298166, ..., 0.00578661, 0.00545532,
        0.0048468 ]])

In [47]:
len(id0_train_array_list)

2294

In [48]:
id0_train_array_list[0].shape

(215, 40)

In [49]:
X_train0_list=[librosa.feature.stack_memory(t.transpose(), n_steps=n_sh) for t in id0_train_array_list]

In [50]:
X_train0 = np.hstack(X_train0_list)

In [51]:
X_train0.shape

(160, 493210)

In [52]:
id1_train_array_list = []
for index, row in id1_train.iterrows(): 
    wavfile = path + os.sep + str(row["itemid"]) + ".wav"
    wavarray, sr = librosa.core.load(wavfile, mono=True)
    if (wavarray.ndim==2): 
        wavarray = np.mean( wavarray, axis=-1 )  
    [fq, t, X] = signal.spectral.spectrogram( wavarray, window=ham_win, nperseg=win, noverlap=0, detrend=False, return_onesided=True, mode='magnitude' ) 
    X = X.T
    X = np.dot( X, melW.T )
    X=X/np.max(X) 
    id1_train_array_list.append(X)

In [53]:
X_train1_list=[librosa.feature.stack_memory(t.transpose(), n_steps=n_sh) for t in id1_train_array_list]

In [54]:
X_train1 = np.hstack(X_train1_list)

In [55]:
X_train1.shape

(160, 168130)

In [56]:
X_train = np.hstack((X_train0, X_train1))

In [57]:
X_train.shape

(160, 661340)

In [58]:
k0 = 50
k1 = 10
M =  X_train.shape[0]
N =  X_train.shape[1]
y0 = X_train0.shape[1]
y1 = X_train1.shape[1]

In [59]:

eps = np.spacing(1)
iterations = 200
np.random.seed(1515)

In [60]:
W = np.random.rand(M, k0+k1) + eps
A = np.block([[np.ones([k0, y0]), np.ones([k0, y1])],
          [np.zeros([k1, y0]), np.ones([k1, y1])]])
H = np.random.rand(k0+k1, y0+y1) + eps
H = A*H
I = np.ones([M,N])
lam = 10

In [61]:
#euclidean cost function
# for i in range(iterations):
#         R = np.dot(W,H)
#         W *= np.dot(X_train , H.T) / (np.dot(I, H.T) + eps)
#         W = W/(np.sum(W,axis=0))
#         H *= np.dot(W.T, X_train) / (np.dot(W.T, I) + eps+10)
#         H = np.divide(H,(np.sum(H,axis=1)).reshape(-1,1))
#         # error = np.linalg.norm(X_train - np.dot(W,H))
#         # print(error)

In [62]:
for i in range(iterations):
        R = np.dot(W,H)
        W *= np.dot(np.divide(X_train, R + eps) , H.T) / (np.dot(I, H.T) + eps)
        W = W/(np.sum(W,axis=0))
        R = np.dot(W,H)
        H *= np.dot(W.T, np.divide(X_train, R + eps)) / (np.dot(W.T, I) + lam + eps)
        H = np.divide(H,(np.sum(H,axis=1)).reshape(-1,1))
        R = np.dot(W,H)
        # error = np.sum(np.multiply(X_train,np.log(((X_train+eps)/(R+eps)))) - X_train + R)
        # print(error)

In [63]:
W.shape

(160, 60)

In [64]:
W

array([[1.37975947e-02, 1.27605963e-05, 3.06950400e-06, ...,
        1.42758197e-05, 3.09369937e-04, 1.00014665e-03],
       [5.02920262e-02, 1.23064144e-05, 1.84698163e-05, ...,
        1.04153560e-05, 8.78051390e-04, 1.80357589e-03],
       [1.01537814e-01, 1.18189467e-04, 5.57940068e-04, ...,
        2.10900398e-05, 1.05543028e-03, 9.98862760e-03],
       ...,
       [1.24792214e-07, 2.80993536e-03, 2.16652606e-06, ...,
        2.19332444e-02, 4.04963256e-04, 1.26053590e-06],
       [5.62806807e-08, 1.53553522e-03, 1.90885917e-07, ...,
        1.23024401e-02, 3.36044027e-04, 2.97958743e-08],
       [2.99509228e-08, 1.36646332e-03, 2.14493949e-07, ...,
        7.32907237e-03, 2.56928000e-04, 4.01773562e-07]])

In [65]:
train_label = data_train['hasbird'].to_numpy().reshape(-1,1)

In [66]:
train_label.shape

(3076, 1)

In [67]:
id_labels = np.vstack((id0_train_labels, id1_train_labels))

In [68]:
I.shape

(160, 661340)

In [69]:
W.shape

(160, 60)

In [70]:
activations = []
for V in X_train0_list:
  I = np.ones(V.shape)
  H = np.random.rand(k0+k1, V.shape[1])
  for i in range(50):
        # H *= np.dot(W.T, V) / (np.dot(W.T, S) + eps)
        R = np.dot(W, H)
        H *= np.dot(W.T, np.divide(V, R + eps)) / (np.dot(W.T, I) + lam + eps)
        H = np.divide(H,(np.sum(H,axis=1)).reshape(-1,1))
  activations.append(np.hstack((np.mean(H, axis = 1).reshape(1,-1), np.std(H, axis = 1).reshape(1,-1))))

In [71]:
for V in X_train1_list:
  I = np.ones(V.shape)
  H = np.random.rand(k0+k1, V.shape[1])
  for i in range(50):
        # H *= np.dot(W.T, V) / (np.dot(W.T, S) + eps)
        R = np.dot(W, H)
        H *= np.dot(W.T, np.divide(V, R + eps)) / (np.dot(W.T, I) + lam + eps)
        H = np.divide(H,(np.sum(H,axis=1)).reshape(-1,1))
  activations.append(np.hstack((np.mean(H, axis = 1).reshape(1,-1), np.std(H, axis = 1).reshape(1,-1))))

In [72]:
X_clf = np.array(activations).squeeze()
X_clf.shape

(3076, 120)

In [73]:

n_trees = 500
clf =ensemble.RandomForestClassifier(n_trees, n_jobs=-1)

In [74]:
clf.fit(X_clf, id_labels.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [75]:
data_test_labels = data_test.iloc[:,1].values.reshape(-1,1)

In [76]:
data_test_array_list = []
for index, row in data_test.iterrows(): 
    wavfile = path + os.sep + str(row["itemid"]) + ".wav"
    wavarray, sr = librosa.core.load(wavfile, mono=True)
    if (wavarray.ndim==2): 
        wavarray = np.mean( wavarray, axis=-1 )    
    [fq, t, X] = signal.spectral.spectrogram( wavarray, window=ham_win, nperseg=win, noverlap=0, detrend=False, return_onesided=True, mode='magnitude' ) 
    X = X.T
    X = np.dot( X, melW.T )
    X=X/np.max(X) 
    data_test_array_list.append(X)

In [77]:
data_test_array_list[0].shape

(215, 40)

In [78]:
data_test_list=[librosa.feature.stack_memory(t.transpose(), n_steps=n_sh) for t in data_test_array_list]

In [79]:
test_activations = []
for V in data_test_list:
  I = np.ones(V.shape)
  H = np.random.rand(k0+k1, V.shape[1])
  for i in range(50):
        # H *= np.dot(W.T, V) / (np.dot(W.T, S) + eps)
        R = np.dot(W, H)
        H *= np.dot(W.T, np.divide(V, R + eps)) / (np.dot(W.T, I) + lam + eps)
        H = np.divide(H,(np.sum(H,axis=1)).reshape(-1,1))
  test_activations.append(np.hstack((np.mean(H, axis = 1).reshape(1,-1), np.std(H, axis = 1).reshape(1,-1))))

In [80]:
X_clf_test = np.array(test_activations).squeeze()
X_clf_test.shape

(769, 120)

In [81]:
y_scores=clf.predict_proba(X_clf_test)

In [82]:
clf_labels = np.argmax(y_scores, axis = 1)

In [83]:
fpr, tpr, thresholds = metrics.roc_curve(data_test_labels, clf_labels)
roc_auc = metrics.auc(fpr, tpr)
print(roc_auc)

0.7237643840784759


In [84]:
print(metrics.accuracy_score(data_test_labels, clf_labels))

0.8634590377113134


In [86]:
print(metrics.confusion_matrix(data_test_labels, clf_labels))

[[581   8]
 [ 97  83]]
